# Handwritten Digit Recognition

Handwritten Digit Recognition using Convolutional Neural Networks in Python with Keras

In [ ]:
import numpy as np
from keras.models import load_model
from matplotlib import pyplot as plt
import cv2

In [ ]:
DEBUG = True

In [ ]:
model = load_model('final_model.h5')

In [ ]:
image_path="imgs/test1.jpg"

# Read the input image 
img = cv2.imread(image_path)

if DEBUG:
    plt.imshow(img)
    plt.title('Original')
    plt.show()

# Convert to grayscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

if DEBUG:
    plt.imshow(img_gray, cmap='gray')
    plt.title('Grayscale')
    plt.show()

# Apply Gaussian filtering
img_gau = cv2.GaussianBlur(img_gray, (5, 5), 0)

if DEBUG:
    plt.imshow(img_gau, cmap='gray')
    plt.title('GaussianBlur')
    plt.show()

# Threshold the image
ret, img_th = cv2.threshold(img_gau, 80, 255, cv2.THRESH_BINARY_INV)

if DEBUG:
    plt.imshow(img_th, cmap='gray')
    plt.title('Binary Image')
    plt.show()

# Find contours in the image
im2, contours, hierarchy = cv2.findContours(img_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if DEBUG:
    print('Contours:', len(contours))

In [ ]:
def get_square(image, square_size):
    height, width = image.shape
    
    if height > width:
        differ = height
    else:
        differ = width
        
    differ += differ // 2

    mask = np.zeros((differ, differ), dtype='uint8')  
    
    x_pos = int((differ - width) / 2)
    y_pos = int((differ - height) / 2)
    
    mask[y_pos:y_pos+height,x_pos:x_pos+width] = image
    mask = cv2.resize(mask, (square_size, square_size), interpolation=cv2.INTER_AREA)

    return mask

In [ ]:
def put_text_with_bg(img, text, x, y, bg=(0, 255, 0)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2

    # Get the width and height of the text box
    text_width, text_height = cv2.getTextSize(text, font, font_scale, thickness)[0]

    # Make the coords of the box with a small padding of two pixels
    box_coords = ((x, y), (x + text_width - 2, y - text_height - 2))
    
    cv2.rectangle(img, box_coords[0], box_coords[1], bg, cv2.FILLED)
    cv2.putText(img, text, (x, y), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)

In [ ]:
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)

    if w > 50 or h > 50:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Region of interest
        roi = img_th[y:y+h, x:x+w]
        
        # Make the rectangular region around the digit
        roi = get_square(roi, 28)
        
        # Dilate the image
        roi = cv2.dilate(roi, (3, 3))
        
        if DEBUG:
            plt.imshow(roi, cmap='gray')
            plt.show()
        
        # Convert to numpy array
        roi = np.array([roi])
        # Reshape to have a single channel
        roi = roi.reshape(1, 28, 28, 1)
        
        # Convert from integers to floats
        roi = roi.astype('float32')
        # Normalize to range 0-1
        roi = roi / 255.
        
        # Predict the class
        digit = model.predict_classes(roi)
        
        put_text_with_bg(img, str(digit[0]), x - 1, y - 1)

        if DEBUG:
            print('Rect:', x, y, w, h)
            print('Digit:', digit[0])

In [ ]:
# Save final image
cv2.imwrite('output1.jpg', img)

In [ ]:
# Show final image
plt.figure(figsize=(20, 20))
plt.imshow(img)
plt.title('Final Image')
plt.show()